# 🧠 Generate Images with StyleGAN2-ADA-pytorch

In [1]:
import sys
sys.path.insert(0, '/home/s25vargason1/richard/512_basic_training/stylegan2-ada-pytorch')

import os
import pickle
import torch
import numpy as np
from torchvision.utils import save_image
import dnnlib
import legacy

## Step 3 - Configuration

In [2]:
# Path to your trained or pretrained .pkl file
pkl_path = '/home/s25vargason1/richard/512_basic_training/stylegan2-ada-pytorch/results/00008-post-impressionist-auto1-ada-resumecustom/network-snapshot-009410.pkl'  # <<< Change this!

# Where to save generated images
output_dir = './generated_images'

# Number of images to generate
num_images = 1

# Truncation psi (lower = higher quality, but less diversity)
truncation_psi = 0.7

# Random seed for reproducibility
random_seed = 42

# Set device (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Set seeds
np.random.seed(random_seed)
torch.manual_seed(random_seed)

## Step 4 - Load the Generator Network

In [3]:
print(f'Loading network from {pkl_path}...')

# Load model first
with open(pkl_path, 'rb') as f:
    G = legacy.load_network_pkl(f)['G_ema']

# Fix precision
if device.type == 'cpu':
    G = G.float()

# Now move to CPU or GPU
G = G.to(device)

print('✅ Network loaded successfully and ready for generation!')

Loading network from /home/s25vargason1/richard/512_basic_training/stylegan2-ada-pytorch/results/00008-post-impressionist-auto1-ada-resumecustom/network-snapshot-009410.pkl...
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
✅ Network loaded successfully and ready for generation!


## Step 5 - Generate and Save Images

In [4]:
for i in range(num_images):
    # Sample a random latent vector z
    z = torch.from_numpy(np.random.randn(1, G.z_dim)).to(device)
    c = None  # No class labels (for unconditional models)

    # Generate image
    img = G(z, c, truncation_psi=truncation_psi, noise_mode='const')

    # Post-process image
    img = (img + 1) * (255/2)
    img = img.clamp(0, 255).to(torch.uint8)
    img = img[0].permute(1, 2, 0).cpu()  # CHW -> HWC

    # Save image
    save_path = os.path.join(output_dir, f'image_{i:04d}.png')
    save_image(img.permute(2, 0, 1).float() / 255.0, save_path)

    print(f'Saved {save_path}')

print('✅ Done generating images!')

RuntimeError: "slow_conv_transpose2d_out_cpu" not implemented for 'Half'

## ✨ Optional Step - Preview Generated Images

In [ ]:
from IPython.display import Image, display
import glob

# Preview some generated images
generated_imgs = sorted(glob.glob(os.path.join(output_dir, '*.png')))

for img_path in generated_imgs[:5]:  # Show first 5 images
    display(Image(filename=img_path))